# Assignment2
date: 19-01-2025

## Main program

In [9]:
import numpy as np
n = 6
npart = n**3
spacing = 1.0
rc = 2.5    # cutoff lenth
coords = np.array([[i, j, k] for i in range(n) for j in range(n) for k in range(n)])*spacing
print('Number of particles = ',npart)
print(coords[215])
coords.shape

Number of particles =  216
[5. 5. 5.]


(216, 3)

In [10]:
# All functons used in this cell are written below(RUN THOSE CELL FIRST!)
box_length = 6.348475790538236
newcoords = PBC(coords,box_length)
F, total_pot = forces(coords)
print('total potential energy = ',total_pot)
coords.shape

r cutoff  =   2.5
total potential energy =  -614.1894071956373


(216, 3)

### Implementation of periodic boundary conditions


In [6]:
def PBC(coord,box_length):
    """
    PBC is applied in this function.
    coord: 2D array of size (particle no. , 3)
    box_lenth: lenth of one side of cubic box
    """
    coord = np.where(coord > length, coord - length, coord)
    coord = np.where(coord < 0, coord + length, coord) 
    return coord
    

### Implementation of minimum image convention

In [7]:
def min_img_conv(coordinate1, coordinate2, box_length):
    dx,dy,dz = (coordinate2[0]-coordinate1[0]),(coordinate2[1]-coordinate1[1]),(coordinate2[2]-coordinate1[2])
    dx = dx - round(dx / box_length) * box_length
    dy = dy - round(dy / box_length) * box_length
    dz = dz - round(dz / box_length) * box_length
    return np.array([dx,dy,dz])

### A funtion to calculate the forces on each particle

In [8]:
def forces(coordinates):
    """
    This function is for calculate total force on each particles and total potential energy.
    Input: coordinates(particle no. , 3)
    Output: Force(Particle no.) and Total pot
    """
    print('r cutoff  =  ', rc)
    rc2 = rc**2
    x,y,z = coordinates[:,0],coordinates[:,1],coordinates[:,2]
    npart=len(x)
    F=np.zeros([npart,3])
    total_pot = 0.0
    r2c = 1/rc2
    r6c = r2c**3
    pot_cut = 4.0*r6c*(r6c-1)
    with open("force.txt", "w") as f:
        f.write(f"fx \t \t fy \t \t fz \n")
            
    for i in range(npart-1):
        for j in range(i+1,npart):
            dx,dy,dz = min_img_conv(coordinates[i,:], coordinates[j,:], box_length)
            r2 = dx**2+dy**2+dz**2
            if (r2 < rc2) :
                r2i = 1/r2
                r6i = r2i**3
                ff = 48.0*r2i*r6i*(r6i - 0.5)
                F[i,0] = F[i,0] + ff*dx
                F[j,0] = F[j,0] - ff*dx
                F[i,1] = F[i,1] + ff*dy
                F[j,1] = F[j,1] - ff*dy
                F[i,2] = F[i,2] + ff*dz
                F[j,2] = F[j,2] - ff*dz
                total_pot = total_pot + 4.0*r6i*(r6i-1)- pot_cut
    for i in range(npart):
        with open("force.txt", "a") as f:
            f.write(f"{F[i,0]:.8f}\t{F[i,1]:.8f}\t{F[i,2]:.8f} \n")
    return F, total_pot
            